In [38]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from ema_workbench.util import utilities

In [39]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

1.5.3
3.0


In [40]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator,
)
from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time

ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

In [41]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)




CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [42]:
### print uncertainties
#
from prettytable import PrettyTable

# Create a new table
table = PrettyTable()
table.field_names = ["Parameter", "Lower Bound", "Upper Bound"]

# Add rows to the table
for uncertainty in uncertainties:
    table.add_row([uncertainty.name, uncertainty.lower_bound, uncertainty.upper_bound])

# Print the table
print(table)

# #Print it simple
# for uncertainty in uncertainties:
#     print("Parameter: {}".format(uncertainty.name))
#     print("Range: [{}, {}]".format(uncertainty.lower_bound, uncertainty.upper_bound))
#     print("---------------------------")

+-------------------------+-------------+-------------+
|        Parameter        | Lower Bound | Upper Bound |
+-------------------------+-------------+-------------+
|     discount rate 0     |      0      |      3      |
|     discount rate 1     |      0      |      3      |
|     discount rate 2     |      0      |      3      |
| A.0_ID flood wave shape |      0      |     132     |
|         A.1_Bmax        |      30     |     350     |
|        A.1_pfail        |      0      |      1      |
|        A.1_Brate        |      0      |      2      |
|         A.2_Bmax        |      30     |     350     |
|        A.2_pfail        |      0      |      1      |
|        A.2_Brate        |      0      |      2      |
|         A.3_Bmax        |      30     |     350     |
|        A.3_pfail        |      0      |      1      |
|        A.3_Brate        |      0      |      2      |
|         A.4_Bmax        |      30     |     350     |
|        A.4_pfail        |      0      |      1

In [43]:
from ema_workbench import IntegerParameter

def get_levers(model):
    levers = []

    for lever in model.levers:
        if isinstance(lever, IntegerParameter):
            levers.append(lever.name)

    return levers

# Example usage
model, _ = get_model_for_problem_formulation(3)
lever_list = get_levers(model)

print(lever_list)


['0_RfR 0', '0_RfR 1', '0_RfR 2', '1_RfR 0', '1_RfR 1', '1_RfR 2', '2_RfR 0', '2_RfR 1', '2_RfR 2', '3_RfR 0', '3_RfR 1', '3_RfR 2', '4_RfR 0', '4_RfR 1', '4_RfR 2', 'EWS_DaysToThreat', 'A.1_DikeIncrease 0', 'A.1_DikeIncrease 1', 'A.1_DikeIncrease 2', 'A.2_DikeIncrease 0', 'A.2_DikeIncrease 1', 'A.2_DikeIncrease 2', 'A.3_DikeIncrease 0', 'A.3_DikeIncrease 1', 'A.3_DikeIncrease 2', 'A.4_DikeIncrease 0', 'A.4_DikeIncrease 1', 'A.4_DikeIncrease 2', 'A.5_DikeIncrease 0', 'A.5_DikeIncrease 1', 'A.5_DikeIncrease 2']


In [44]:
def get_levers_with_ranges(model):
    levers = []

    for lever in model.levers:
        if isinstance(lever, IntegerParameter):
            lever_range = (lever.lower_bound, lever.upper_bound)
            levers.append((lever.name, lever_range))

    return levers

# Example usage
model, _ = get_model_for_problem_formulation(3)
levers_with_ranges = get_levers_with_ranges(model)

for lever_name, lever_range in levers_with_ranges:
    print(f"Lever: {lever_name}")
    print(f"Range: {lever_range}")
    print()


Lever: 0_RfR 0
Range: (0, 1)

Lever: 0_RfR 1
Range: (0, 1)

Lever: 0_RfR 2
Range: (0, 1)

Lever: 1_RfR 0
Range: (0, 1)

Lever: 1_RfR 1
Range: (0, 1)

Lever: 1_RfR 2
Range: (0, 1)

Lever: 2_RfR 0
Range: (0, 1)

Lever: 2_RfR 1
Range: (0, 1)

Lever: 2_RfR 2
Range: (0, 1)

Lever: 3_RfR 0
Range: (0, 1)

Lever: 3_RfR 1
Range: (0, 1)

Lever: 3_RfR 2
Range: (0, 1)

Lever: 4_RfR 0
Range: (0, 1)

Lever: 4_RfR 1
Range: (0, 1)

Lever: 4_RfR 2
Range: (0, 1)

Lever: EWS_DaysToThreat
Range: (0, 4)

Lever: A.1_DikeIncrease 0
Range: (0, 10)

Lever: A.1_DikeIncrease 1
Range: (0, 10)

Lever: A.1_DikeIncrease 2
Range: (0, 10)

Lever: A.2_DikeIncrease 0
Range: (0, 10)

Lever: A.2_DikeIncrease 1
Range: (0, 10)

Lever: A.2_DikeIncrease 2
Range: (0, 10)

Lever: A.3_DikeIncrease 0
Range: (0, 10)

Lever: A.3_DikeIncrease 1
Range: (0, 10)

Lever: A.3_DikeIncrease 2
Range: (0, 10)

Lever: A.4_DikeIncrease 0
Range: (0, 10)

Lever: A.4_DikeIncrease 1
Range: (0, 10)

Lever: A.4_DikeIncrease 2
Range: (0, 10)

Lever: 

In [45]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [46]:
#enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('A.1 Total Costs', variable_name=('A.1_Expected Annual Damage', 'A.1_Dike Investment Costs'), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.1_Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths',), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.2 Total Costs', variable_name=('A.2_Expected Annual Damage', 'A.2_Dike Investment Costs'), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.2_Expected Number of Deaths', variable_name=('A.2_Expected Number of Deaths',), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.3 Total Costs', variable_name=('A.3_Expected Annual Damage', 'A.3_Dike Investment Costs'), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.3_Expected Number of Deaths', variable_name=('A.3_Expected Number of Deaths',), function=<function sum_over at 0x0000017717687DC0>)
ScalarOutcome('A.4 Total Costs', variable_name=('A.4_Expected Annual Dama

In [47]:
ema_logging.log_to_stderr(ema_logging.INFO)


with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=50, policies=4)

print(type(results))

[MainProcess/INFO] pool started with 12 workers
[MainProcess/INFO] performing 50 scenarios * 4 policies * 1 model(s) = 200 experiments
100%|████████████████████████████████████████| 200/200 [00:20<00:00,  9.56it/s]
[MainProcess/INFO] experiments finished
[MainProcess/INFO] terminating pool


<class 'tuple'>


In [66]:
# WERKT NOG NIEEEET
#Analyzing lever interactions
from ema_workbench.analysis import prim

ema_logging.log_to_stderr(ema_logging.INFO)

# Extracting outcomes from the results
experiments, outcomes = results

# Creating a DataFrame from the experiments
df_experiments = pd.DataFrame(experiments)

print(f"Shape of df_experiments: {df_experiments.shape[0]} rows, {df_experiments.shape[1]} columns.")
df_experiments.head(3)

# Creating a DataFrame for the outcomes
df_outcomes = pd.DataFrame(outcomes)

print(f"Shape of df_outcomes: {df_outcomes.shape[0]} rows, {df_outcomes.shape[1]} columns.")
df_outcomes.head(3)


# Defining the levers you want to analyze
# levers_to_analyze = ["A.5_DikeIncrease 0", "A.EWS_DaysToThreat", "4_RfR 2"]

# Performing the PRIM analysis
prim_obj = prim.Prim(df_experiments, df_outcomes, threshold=0.2)
box1 = prim_obj.find_box()

# Printing the box statistics
print(box1.stats())

# Visualizing the box
box1.show_tradeoff()
plt.show()



Shape of df_experiments: 200 rows, 53 columns.
Shape of df_outcomes: 200 rows, 12 columns.


NameError: name 'sdutil' is not defined